In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from core.SimpleNet import SimpleNet
from core.OldGraphConvNet import OldGraphConvNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import get_net_projection, neighborhood_preservation, trustworthiness, evaluate_net_metrics
from util.graph_utils import get_shortest_path_matrix

In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [6]:
dataset_name = 'pubmed'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)

In [7]:
dataset.create_all_data(n_batches=50, sampling=True)

Time to create all data (s) = 9.3707


In [8]:
print([len(G.labels) for G in dataset.all_data])

[5012, 5112, 4976, 5089, 4951, 4991, 5211, 5146, 5155, 4946, 5128, 5043, 5129, 5019, 5119, 5075, 5238, 5056, 5031, 4903, 5009, 5031, 5040, 4973, 5130, 4969, 5171, 4989, 5034, 5213, 4976, 5109, 4958, 4850, 4804, 5167, 4911, 4868, 5127, 4968, 5225, 5078, 5001, 4930, 5057, 4886, 4980, 5330, 5231, 4174]


In [9]:
G = dataset.all_data[0]

In [10]:
from core.tsne_torch_loss import compute_joint_probabilities

In [18]:
X = G.data.view(G.data.shape[0], -1).numpy()
import time
t_start = time.time()
P = compute_joint_probabilities(X, perplexity=30, metric='cosine', adj=G.adj_matrix, alpha=0)
print("1. Time to compute P matrix = {}".format(time.time() - t_start))

Time to compute X2P = 6.185444116592407
1. Time to compute P matrix = 7.2952470779418945


In [12]:
from sklearn.manifold.t_sne import _joint_probabilities

In [17]:
distances = pairwise_distances(X, metric='cosine')
t_start = time.time()
new_P = _joint_probabilities(distances, desired_perplexity=30, verbose=1)
print("1. Time to compute P matrix = {}".format(time.time() - t_start))

[t-SNE] Computed conditional probabilities for sample 1000 / 5012
[t-SNE] Computed conditional probabilities for sample 2000 / 5012
[t-SNE] Computed conditional probabilities for sample 3000 / 5012
[t-SNE] Computed conditional probabilities for sample 4000 / 5012
[t-SNE] Computed conditional probabilities for sample 5000 / 5012
[t-SNE] Computed conditional probabilities for sample 5012 / 5012
[t-SNE] Mean sigma: 0.206628
1. Time to compute P matrix = 8.29175615310669


In [ ]:
distances = pairwise_distances(X, metric='cosine')
t_start = time.time()
new_P = _joint_probabilities(distances, desired_perplexity=30, verbose=1)
print("1. Time to compute P matrix = {}".format(time.time() - t_start))

In [14]:
from scipy.spatial.distance import squareform
new_P_2 = squareform(new_P)

In [15]:
new_P_2.shape

(5012, 5012)

In [16]:
sum(sum(new_P_2))

1.0000000000000013